In [1]:
import pandas as pd
import os
import numpy as np
import ast

In [2]:
df = pd.read_csv('NLP_input.csv')
df.head()

,Name,Text Data,depression,flattened text,tokenized_text,expanded_text
0,369,"[['okay', 'awesome', 'thank', 'you'], ['are', ...",0,"['okay', 'awesome', 'thank', 'you', 'are', 'yo...","['okay', 'awesome', 'thank', 'you', 'are', 'yo...","['okay', 'awesome', 'thank', 'you', 'are', 'yo..."
1,423,"[['okay'], ['and', 'please'], ['yes'], ['feeli...",0,"['okay', 'and', 'please', 'yes', 'feeling', 'w...","['okay', 'and', 'please', 'yes', 'feeling', 'w...","['okay', 'and', 'please', 'yes', 'feeling', 'w..."
2,436,"[['that', 'was', 'big'], ['yes'], [""I'm"", 'doi...",0,"['that', 'was', 'big', 'yes', ""I'm"", 'doing', ...","['that', 'was', 'big', 'yes', 'I', ""'m"", 'doin...","['that', 'was', 'big', 'yes', 'I', 'am', 'doin..."
3,318,"[['all', 'right'], ['okay'], ['oh', 'actually'...",0,"['all', 'right', 'okay', 'oh', 'actually', 'be...","['all', 'right', 'okay', 'oh', 'actually', 'be...","['all', 'right', 'okay', 'oh', 'actually', 'be..."
4,447,"[['yeah', ""that's"", 'perfectly', 'fine'], [""I'...",0,"['yeah', ""that's"", 'perfectly', 'fine', ""I'm"",...","['yeah', 'that', ""'s"", 'perfectly', 'fine', 'I...","['yeah', 'that', 'is', 'perfectly', 'fine', 'I..."


In [3]:
df=df.drop(columns = ['tokenized_text','expanded_text','flattened text'])

In [4]:
import ast
columns = ['Text Data']

for item in columns:
    df[item] = df[item].apply(lambda x: ast.literal_eval(x))

In [5]:
df.head()

,Name,Text Data,depression
0,369,"[[okay, awesome, thank, you], [are, you, okay,...",0
1,423,"[[okay], [and, please], [yes], [feeling, well]...",0
2,436,"[[that, was, big], [yes], [I'm, doing, fine], ...",0
3,318,"[[all, right], [okay], [oh, actually, before, ...",0
4,447,"[[yeah, that's, perfectly, fine], [I'm, feelin...",0


In [6]:
def process_text_data(text_data):
    # For each sublist, join the words in lowercase with spaces and add a period
    return [' '.join(word.lower() for word in sublist) + '.' for sublist in text_data]

# Apply the function to each entry in the 'Text Data' column
df['Processed Text Data'] = df['Text Data'].apply(process_text_data)
df['Processed Text Data'] = df['Processed Text Data'].apply(lambda x: ' '.join(x))

In [7]:
df.head()

,Name,Text Data,depression,Processed Text Data
0,369,"[[okay, awesome, thank, you], [are, you, okay,...",0,okay awesome thank you. are you okay with this...
1,423,"[[okay], [and, please], [yes], [feeling, well]...",0,okay. and please. yes. feeling well. where are...
2,436,"[[that, was, big], [yes], [I'm, doing, fine], ...",0,that was big. yes. i'm doing fine. mexico. whe...
3,318,"[[all, right], [okay], [oh, actually, before, ...",0,all right. okay. oh actually before that one t...
4,447,"[[yeah, that's, perfectly, fine], [I'm, feelin...",0,yeah that's perfectly fine. i'm feeling great ...


In [ ]:
import random
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import StratifiedKFold, train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
from time import sleep
from sklearn.metrics import f1_score
from tensorflow.keras.callbacks import EarlyStopping
import transformers


2024-11-25 13:30:37.847348: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-25 13:30:37.847408: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-25 13:30:37.849235: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-25 13:30:37.858542: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-25 13:30:38.855499: W tensorflow/compiler/tf2

In [13]:
k = 2
seed_values = list(range(6,7))

for seed in seed_values:
    print(f"Running with seed: {seed}")
    data = df
    # Set the seed for Python random module
    random.seed(seed)

    # Set the seed for NumPy
    np.random.seed(seed)
    # Set the seed for TensorFlow
    tf.random.set_seed(seed)
    # Initialize the stratified k-fold cross-validator
    kf = StratifiedKFold(n_splits=k,random_state=seed, shuffle=True) # ensure the shuffling is the reproducible
    # Load the pre-trained BERT model and tokenizer
    model_name = "bert-base-uncased"
    tokenizer = BertTokenizer.from_pretrained(model_name)
    # Define a loss function and metrics
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    metrics = [tf.keras.metrics.SparseCategoricalAccuracy("accuracy")]
    # Define an optimizer
    early_stopping = EarlyStopping(
    monitor="val_loss",  # Monitor validation loss
    patience=5,          # Stop training after 3 epochs of no improvement
    restore_best_weights=True  # Restore model weights from the epoch with the best validation performance
    )
    # optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
    # optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
    # tf.keras.optimizers.Optimizer(
    # name='adam',
    # learning_rate=0.001,
    # weight_decay=0,
    # clipnorm=None,
    # clipvalue=None,
    # global_clipnorm=None,
    # use_ema=False,
    # ema_momentum=0.99,
    # ema_overwrite_frequency=None,
    # jit_compile=True,
    # )

    # Split the dataset into features and target arrays
    texts = df['Processed Text Data'].values
    labels = data['depression'].values
    fold_results = []
    fold_f1_scores = []
    # i=1
    # Iterate through the k folds

    for fold, (train_val_index, test_index) in enumerate(kf.split(texts, labels)):
      # Set the seed for each fold (to ensure the reproducibility of each fold, these are fold-specific seeds)
      tf.random.set_seed(seed)
      np.random.seed(seed)
      random.seed(seed)
      # Split train_val set to train and validation sets in 8:2 ratio
      train_index, valid_index = train_test_split(train_val_index, test_size=0.2, stratify=labels[train_val_index], random_state=seed) #ensure the splitting is reproducible
      print(f"Fold {fold + 1}/{k}")
      # Create train, validation, and test datasets for this fold
      train_texts, train_labels = texts[train_index], labels[train_index]
      valid_texts, valid_labels = texts[valid_index], labels[valid_index]
      test_texts, test_labels = texts[test_index], labels[test_index]
      # Tokenize the input texts for the fold
      train_inputs = tokenizer(list(train_texts), padding=True, truncation=True, return_tensors="tf")
      valid_inputs = tokenizer(list(valid_texts), padding=True, truncation=True, return_tensors="tf")
      test_inputs = tokenizer(list(test_texts), padding=True, truncation=True, return_tensors="tf")
      # Define TensorFlow Datasets for training and validation
      train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_inputs), train_labels))
      valid_dataset = tf.data.Dataset.from_tensor_slices((dict(valid_inputs), valid_labels))
      test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_inputs), test_labels))
      # Compile the model for this fold
      model.compile(optimizer=opt, loss=loss_fn, metrics=metrics)
      # Train the model for this fold
      batch_size = 8
      num_epochs = 20
      model.fit(train_dataset.shuffle(100).batch(batch_size), epochs=num_epochs, validation_data=valid_dataset.batch(batch_size),callbacks=[early_stopping])
      # Evaluate the model on the validation data for this fold
      loss, accuracy = model.evaluate(valid_dataset.batch(batch_size))
      print(f"Validation Loss: {loss}, Validation Accuracy: {accuracy}")

      valid_pred_probs = model.predict(valid_dataset.batch(batch_size))
      valid_pred_labels = np.argmax(valid_pred_probs.logits, axis=1)
      # Compute F1 score for this fold
      f1 = f1_score(valid_labels, valid_pred_labels, average='weighted')
      print(f"Validation F1 Score for Fold {fold + 1}: {f1}")
      # Store the F1 score for this fold
      fold_f1_scores.append(f1)

      model_save_path = f"../sample_data/model/model{fold + 1}"
      model.save_pretrained(model_save_path)
      # Create a DataFrame for test data
      test_data_df = pd.DataFrame({'Case':test_index,'text': test_texts, 'label': test_labels})
      # Save the test data to a CSV file
      test_data_csv_path = f"../sample_data/dataset/test{fold + 1}.csv"
      test_data_df.to_csv(test_data_csv_path, index=False)
    sleep(2)

    final_results = []
    for fold in range(k):
        # Load the model for the current fold
        model_path = f"../sample_data/model/model{fold + 1}"
        loaded_model = TFBertForSequenceClassification.from_pretrained(model_path)
        # Load the test data for the current fold
        data_path = f"../sample_data/dataset/test{fold + 1}.csv"
        data = pd.read_csv(data_path)

        # Prepare to store results
        predictions = []
        predicted_probabilities_list0 = []
        predicted_probabilities_list1 = []
        # predicted_probabilities_list2 = []
        # predicted_probabilities_list3 = []
        # Iterate over each row in the data
        for text in data['text']:
            test_inputs = tokenizer(text, padding=True, truncation=True, return_tensors="tf")
            prediction = loaded_model(test_inputs)
            predicted_probabilities = tf.nn.softmax(prediction.logits, axis=1).numpy()
            predicted_class = np.argmax(predicted_probabilities, axis=1)

            predictions.append(predicted_class[0])
            predicted_probabilities_list0.append(predicted_probabilities[0][0])
            predicted_probabilities_list1.append(predicted_probabilities[0][1])
            # predicted_probabilities_list2.append(predicted_probabilities[0][2])
            # predicted_probabilities_list3.append(predicted_probabilities[0][3])
        # Store results in the DataFrame
        data['predicted_class'] = predictions
        data['Fold_Model'] = fold + 1
        data['prob_class_0'] = predicted_probabilities_list0
        data['prob_class_1'] = predicted_probabilities_list1
        # data['prob_class_2'] = predicted_probabilities_list2
        # data['prob_class_3'] = predicted_probabilities_list3

        # Append the results of this fold to the final results
        final_results.append(data)

    # Concatenate all results into a single DataFrame
    final_result_data = pd.concat(final_results)
    sleep(2)
    final_result_data.to_csv(f"../sample_data/dataset/result_depression_epoch20.csv")

Running with seed: 6


/data0/home/h24/chso7162/myenv/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Fold 1/2


/data0/home/h24/chso7162/myenv/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/20
11/11 [==============================] - 145s 11s/step - loss: 0.6506 - accuracy: 0.6552 - val_loss: 0.5916 - val_accuracy: 0.7273
Epoch 2/20
11/11 [==============================] - 111s 10s/step - loss: 0.6170 - accuracy: 0.7011 - val_loss: 0.5894 - val_accuracy: 0.7273
Epoch 3/20
11/11 [==============================] - 112s 10s/step - loss: 0.5947 - accuracy: 0.7011 - val_loss: 0.5859 - val_accuracy: 0.7273
Epoch 4/20
11/11 [==============================] - 112s 10s/step - loss: 0.5982 - accuracy: 0.7011 - val_loss: 0.5904 - val_accuracy: 0.7273
Epoch 5/20
11/11 [==============================] - 112s 10s/step - loss: 0.5635 - accuracy: 0.7011 - val_loss: 0.5944 - val_accuracy: 0.7273
Epoch 6/20
11/11 [==============================] - 111s 10s/step - loss: 0.5020 - accuracy: 0.8046 - val_loss: 0.6307 - val_accuracy: 0.6818
Epoch 7/20
11/11 [==============================] - 110s 10s/step - loss: 0.4568 - accuracy: 0.8391 - val_loss: 0.6340 - val_accuracy: 0.6818
Epoch 

/data0/home/h24/chso7162/myenv/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/20
11/11 [==============================] - 140s 11s/step - loss: 0.6532 - accuracy: 0.6636 - val_loss: 0.6284 - val_accuracy: 0.6818
Epoch 2/20
11/11 [==============================] - 110s 10s/step - loss: 0.6068 - accuracy: 0.7045 - val_loss: 0.6226 - val_accuracy: 0.6818
Epoch 3/20
11/11 [==============================] - 112s 10s/step - loss: 0.5991 - accuracy: 0.7045 - val_loss: 0.6193 - val_accuracy: 0.6818
Epoch 4/20
11/11 [==============================] - 111s 10s/step - loss: 0.5954 - accuracy: 0.7045 - val_loss: 0.6308 - val_accuracy: 0.6818
Epoch 5/20
11/11 [==============================] - 112s 10s/step - loss: 0.5923 - accuracy: 0.7045 - val_loss: 0.6151 - val_accuracy: 0.6818
Epoch 6/20
11/11 [==============================] - 112s 10s/step - loss: 0.5690 - accuracy: 0.7045 - val_loss: 0.6099 - val_accuracy: 0.6818
Epoch 7/20
11/11 [==============================] - 112s 10s/step - loss: 0.5326 - accuracy: 0.7045 - val_loss: 0.6221 - val_accuracy: 0.6818
Epoch 

Some layers from the model checkpoint at ../sample_data/model/model1 were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at ../sample_data/model/model1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.
Some layers from the model checkpoint at ../sample_data